In [1]:
import asyncio
from datetime import datetime as dt, timedelta

from nab3 import AWS as NabAWS
from nab3.helpers.cloud_watch import md_statistics_summary
from nab3.utils import Filter

AWS = NabAWS()
ECS_CLUSTERS = await AWS.ecs_cluster.list()

In [2]:
f = Filter(name__icontains_any=['sample_cluster_1', 'sample_cluster_2'])
filtered_clusters = await f.run(ECS_CLUSTERS)
print([c.name for c in filtered_clusters])

['houston', 'cheyenne']


In [9]:
MINUTE = 60
HOUR = MINUTE * 60
DAY = HOUR * 24

async def get_cluster_stats(ecs_cluster, start_date=dt.now()-timedelta(days=30), end_date=dt.now()) -> dict:
    async def _stat(metric):
        return dict(
            metric=metric,
            stats=ecs_cluster.get_statistics(metric_name=metric,
                                             statistics=['Average', 'Maximum'],
                                             start_time=start_date,
                                             end_time=end_date,
                                             interval_as_seconds=MINUTE*30)
        )
    stats = await asyncio.gather(*[_stat(metric) for metric in ['CPUUtilization', 'MemoryUtilization']])
    return dict(ecs_cluster=ecs_cluster, stats=stats)


cluster_stats = await asyncio.gather(*[get_cluster_stats(ecs_cluster) for ecs_cluster in filtered_clusters])

In [10]:
md_output = ""
for cluster_dict in cluster_stats:
    cluster = cluster_dict['ecs_cluster']
    md_output += f"#{cluster.name}\n"
    
    for stat_dict in cluster_dict['stats']:
        md_output += md_statistics_summary(stat_dict['stats'], stat_dict['metric'], False)

    md_output += "\n---\n"

with open('cluster_stats.md', 'w') as f:
    f.write(md_output)

